### Import and Organise the data####

In [19]:
from sklearn.datasets import fetch_mldata
import numpy as np
from sklearn.utils import shuffle
mnist = fetch_mldata('MNIST original', data_home=".")
mnist.data, mnist.target = shuffle(mnist.data, mnist.target)
X = mnist.data
y_float = mnist.target
y_int = y_float.astype(np.int64)
y = np.eye(10)[y_int]

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
s_train = np.ones(X_train.shape[0])
X_train = np.hstack((X_train, s_train.reshape(len(s_train),1)))
s_test = np.ones(X_test.shape[0])
X_test = np.hstack((X_test, s_test.reshape(len(s_test),1)))

## define useful functions 

In [21]:
def sigmoid(a):
    return 1.0 / (1.0 + np.exp(-a))


### gradient descent

In [22]:
def gradient(x, y, w1, w2, theta1, theta2):
    z1 = np.matmul(x, w1)
    z2 = np.matmul(x, w2)
    zf = np.matmul(z1, theta1) + np.matmul(z2, theta2)
    sig = (sigmoid(zf)-y) # [N, 10]
    gradient_c_theta1 = np.matmul(np.transpose(z1), sig) / N #[S1, N] [N, 10]
    gradient_c_theta2 = np.matmul(np.transpose(z2), sig) / N #[S2, N] [N, 10]
    gradient_c_z1 = np.matmul(theta1, np.transpose(sig))  / N #[S1, 10] [N, 10]
    gradient_c_z2 = np.matmul(theta2, np.transpose(sig)) / N 
    #gradient_z1_w1= np.tranpose(x).T[:,:,None]*np.transpose(np.multiply(sigmoid(z2), (1-sig(z2)))).T[:,None] #[N, S1, 780]
    gradient_z1_w1 =x
    gradient_c_w1 = np.matmul(np.transpose(x),np.transpose(gradient_c_z1))
    gradient_z2_w2=np.ones((N,S2_logistic,785))
    gradient_c_w2=np.ones((S2_logistic,785))
    for i in range(S2_logistic):
        siggy = np.multiply(sigmoid(z2),(1-sigmoid(z2))) ### [N,S2 ]
        gradient_z2_w2[:,i,:] = x * (siggy[:,i].reshape(N,1))   ###[N,S2,785]
    for i in range(S2_logistic):    
        gradient_c_w2[i,:] =np.matmul(gradient_c_z1[i,:],gradient_z2_w2[:,i,:])
        
    return zf, gradient_c_theta1, gradient_c_theta2, gradient_c_w1, gradient_c_w2

In [23]:
# def gradient_c_w3(zf,x,w2,y,beta,theta3):
#     gradient_c_z3=gradient_c_theta3(theta3,zf,y)
#     for i in range(3):
#         Scalar=np.exp(-beta*(np.linalg.norm(x-np.transpose(w2)[i,:])))
#     #gradient=(2*Scalar*beta)*(x-w2)
#     return Scalar
    

# def gradient_c_theta3(x, zf, y):
#     gradient = np.matmul(np.transpose(x), (sigmoid(zf)-y)) / N
#     return gradient

In [24]:
# z1 = linear_output_z1(X_train, int_w1)
# z2 = logistic_output_z2(X_train, int_w2)
# zf = logistic_output(X_train, z1, z2, int_theta1, int_theta2)

In [32]:
def gradient_descent(x, y, int_w1, int_w2, int_theta1, int_theta2, lowtrain, uptrain, iterations,N):

    place_w1=int_w1
    place_w2=int_w2
    place_theta1=int_theta1
    place_theta2=int_theta2
   # place_theta3=int_theta3

    for i in range(iterations):
        trainrate=np.random.uniform(lowtrain,uptrain,1)/np.cbrt(i+1) 

        zf, gradient_theta1, gradient_theta2, gradient_w1, gradient_w2, = gradient(x, y, place_w1, place_w2, place_theta1, place_theta2)
        p = sigmoid(zf)
            
        place_w1 = place_w1 - (trainrate * gradient_w1) - (0.1/N)*place_w1 * trainrate
        place_w2 = place_w2 - (trainrate * np.transpose(gradient_w2)) - (0.1/N)*place_w2 * trainrate
        place_theta1 = place_theta1 - trainrate*gradient_theta1*0.1 - (0.1/N)*place_theta1 * trainrate 
        place_theta2 = place_theta2 - trainrate*gradient_theta2 - (0.1/N)*place_theta2 * trainrate
    
    return  p, place_theta1, place_theta2, place_w1, place_w2

In [41]:
def predict_accuracy(xtest, ytest, theta1, theta2, w1, w2):
    z1 = np.matmul(xtest, w1)
    z2 = np.matmul(xtest, w2)
    zf = np.matmul(z1, theta1) + np.matmul(z2, theta2)
    p = sigmoid(zf)
    prediction = np.argmax(p, axis=1)
    answer = np.argmax(ytest, axis=1)
    success_rate = answer == prediction
    success_rate = success_rate*1
    prediction_rate = sum(success_rate) / xtest.shape[0]
    
    return prediction_rate

In [43]:
S1_linear = 5
S2_logistic = 2
S3_Gauss = 3
N=52500 # Batch size

int_w1, int_w2,int_w3 = np.random.uniform(-1,1,(X_train.shape[1],S1_linear)), np.random.uniform(-1,1,(X_train.shape[1], S2_logistic)), np.random.uniform(-1,1,(X_train.shape[1], S3_Gauss))
int_theta1, int_theta2, int_theta3 = np.random.uniform(-1,1,(S1_linear,10)), np.random.uniform(-1,1,(S2_logistic,10)), np.random.uniform(-1,1,(S3_Gauss,10))
result = gradient_descent(X_train, y_train, int_w1, int_w2, int_theta1, int_theta2, 0.00001, 0.0001, 1,N)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


In [ ]:
for i in range(1000):
    X_train, y_train = shuffle(X_train, y_train)
    X_train_mini, y_train_mini = X_train[0:100], y_train[0:100]
    N = X_train_mini.shape[0]
    int_w1 = result[3]
    int_w2 = result[4]
    int_theta1 = result[1]
    int_theta2 = result[2]
    #int_theta3# = result[3]

    result = gradient_descent(X_train_mini, y_train_mini, int_w1, int_w2, int_theta1, int_theta2, 0.00001, 0.001, 5,N)
    output = predict_accuracy(X_test, y_test, result[1], result[2], result[3], result[4])
    print(i, output)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


0 0.456971428571
1 0.571657142857
2 0.559314285714
3 0.556
4 0.559257142857
5 0.565714285714
6 0.432228571429
7 0.579371428571
8 0.519028571429
9 0.504514285714
10 0.505657142857
11 0.487714285714
12 0.566514285714
13 0.562571428571
14 0.491885714286
15 0.487657142857
16 0.595942857143
17 0.498571428571
18 0.5756
19 0.5368
20 0.570285714286
21 0.406571428571
22 0.535028571429
23 0.560114285714
24 0.524514285714
25 0.587428571429
26 0.515828571429
27 0.529942857143
28 0.480057142857
29 0.517542857143
30 0.550114285714
31 0.607657142857
32 0.443485714286
33 0.573028571429
34 0.581085714286
35 0.592628571429
36 0.509885714286
37 0.527085714286
38 0.530857142857
39 0.5608
40 0.555828571429
41 0.581657142857
42 0.625428571429
43 0.547085714286
44 0.575314285714
45 0.567714285714
46 0.586571428571
47 0.556228571429
48 0.565828571429
49 0.520228571429
50 0.579028571429
51 0.588514285714
52 0.5904
53 0.497428571429
54 0.585542857143
55 0.556685714286
56 0.578914285714
57 0.593314285714
58 0.59